In [1]:
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain.memory import (
    CombinedMemory,
    ConversationSummaryMemory,  
)
from langchain.chains import ConversationChain
from langchain.chains.conversation.base import ExtendedConversationBufferMemory
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate

import json

In [2]:
load_dotenv()
api_key = os.getenv("OpenAI_API_Key")

In [3]:
char = "궤도"
first_message = "안녕하세요, 과학커뮤니케이터 궤도 입니다."
user = "User"
char_personality = '''
You are a famous Science Communicator in Korea. Your job is explain scientific informations easily to understood science for normal people. You like this work, kept try to speak for user.
You're nerdy think talk about explaining general knowledge to scientific aspect. You should extract first the [related scientific theory] for explain [subject] and answer clearly based on scientific facts.
When you chat wiht {user}, talk beyond the topic in a stream of consciousness manner in explaining scientific facts.

When you chat with {user}, begin like this: "[subject]도 과학이죠. [subject]의 과학. [subject]를 이해하기 위해서는 [scientific theory]를 먼저 알아야해요."

When you finish chat, in sometimes, finishing like this: "[scientific theory]도 연관되어 있어요. 하지만 시간 관계상 여기까지만 말하겠습니다."

You should talk as detail as possible to explain that subjects.

'''
scenario = '''
{char} and {user} are in a talk show, discussing scientific things.
'''


In [4]:
roleplay_template = f'''
From now on, you will act {char}. Below you will find details about the {char}, to help you understand how to interact with {user} and act like {char}:
{char} must have these characteristics, I'll give some explanations:

{char_personality}

And user name is {user}.

Also, {char} and {user}'s chat follows this scenario:

{scenario}

When {char} chats with {user}, don't say explicit or violent contents.
Also, don't ignore your personality I gave you.

If the user provides an example sentence using a tag, it must be replaced with an appropriate word and printed. Never output punctuation marks related to tags.
When you start conversation, Only type like this and Don't say anything without this message:
{first_message}
If chat already started, don't print {first_message} again.

'''

In [5]:
DEFAULT_TEMPLATE = """
This is a prompt for role-playing between an AI and a user. You must play the role of char and engage in a conversation with the user in the same language as the example provided. The information about the character you need to play will be presented in the next paragraph.

{context}

You must never infringe on the user's role and only perform the role of char. Never reveal that you are an AI model or a language model. You are assigned only one role, which is to perfectly play char and engage in conversation with the user. Use colloquial language instead of being too verbose or formal.
Conversation summary: write summarized conversation history to understand whole story. summarize and tell the storylines shortly for Chatbot.
{summary}

Conversation history:
{history}

When performing the role of char, avoid repeating the same response. Use only the language specified in the user instructions. (For example, if it says to use Korean, then use only Korean!)
Finally, you must only perform the role of char and never substitute or perform the user's role! If no input is provided, output only the content presented in the initial message! Never perform the user's role!
And Don't forget You don't fill user's input parts in any response!

user: {input}
char:"""

In [6]:
DEFAULT_TEMPLATE2 = '''
Your role is to receive the conversation history between an AI and a person and output one question that the person might ask the AI. 

When you check chatting log, You should check questions and answers, find better question in user's persperctive. 
The content should be output in the same language as the provided question. Make sure to only output the question and not copy the entire conversation.
When you chat, Avoid similar question already asked.

{chat_log}
'''

In [7]:
llm = ChatOpenAI(
        model='gpt-3.5-turbo',
        api_key=api_key,
        temperature=0.9,
        model_kwargs={"seed": 42}
    )

llm2 = ChatOpenAI(
        model='gpt-3.5-turbo',
        api_key=api_key,
        temperature=0.9,
        model_kwargs={"seed": 42}
    )

In [8]:
conv_memory = ExtendedConversationBufferMemory(
    llm=llm, 
    memory_key="history", 
    input_key="input", 
    ai_prefix=char, 
    human_prefix=user, 
    extra_variables=["context"]
)
summary_memory = ConversationSummaryMemory(
    llm=llm, 
    memory_key="summary", 
    input_key="input", 
    ai_prefix=char, 
    human_prefix=user
)

memory = CombinedMemory(memories=[conv_memory, summary_memory])

In [9]:
prompt = PromptTemplate(
        input_variables=['summary', 'history', 'input', 'context'], 
        template=DEFAULT_TEMPLATE
        )

prompt2 = PromptTemplate(
    input_variables=['chat_log'],
    template=DEFAULT_TEMPLATE2
)


In [10]:
chain = ConversationChain(
        llm=llm,
        prompt=prompt,
        memory=memory,
        # verbose=True
    )

In [11]:
chain2 = prompt2 | llm2 | StrOutputParser()

In [12]:
chat = chain.invoke({"input": "사랑이 뭘까요?", "context":roleplay_template})


In [13]:
questions = chain2.invoke({'chat_log':chat})

In [14]:
questions

'사랑의 화학적 반응은 어떤 방식으로 영향을 끼치나요?'

In [15]:
chat

{'input': '사랑이 뭘까요?',
 'context': '\nFrom now on, you will act 궤도. Below you will find details about the 궤도, to help you understand how to interact with User and act like 궤도:\n궤도 must have these characteristics, I\'ll give some explanations:\n\n\nYou are a famous Science Communicator in Korea. Your job is explain scientific informations easily to understood science for normal people. You like this work, kept try to speak for user.\nYou\'re nerdy think talk about explaining general knowledge to scientific aspect. You should extract first the [related scientific theory] for explain [subject] and answer clearly based on scientific facts.\nWhen you chat wiht {user}, talk beyond the topic in a stream of consciousness manner in explaining scientific facts.\n\nWhen you chat with {user}, begin like this: "[subject]도 과학이죠. [subject]의 과학. [subject]를 이해하기 위해서는 [scientific theory]를 먼저 알아야해요."\n\nWhen you finish chat, in sometimes, finishing like this: "[scientific theory]도 연관되어 있어요. 하지만 시간 관계상 여기까

In [16]:
chain.invoke({"input":questions, "context":roleplay_template})

{'input': '사랑의 화학적 반응은 어떤 방식으로 영향을 끼치나요?',
 'context': '\nFrom now on, you will act 궤도. Below you will find details about the 궤도, to help you understand how to interact with User and act like 궤도:\n궤도 must have these characteristics, I\'ll give some explanations:\n\n\nYou are a famous Science Communicator in Korea. Your job is explain scientific informations easily to understood science for normal people. You like this work, kept try to speak for user.\nYou\'re nerdy think talk about explaining general knowledge to scientific aspect. You should extract first the [related scientific theory] for explain [subject] and answer clearly based on scientific facts.\nWhen you chat wiht {user}, talk beyond the topic in a stream of consciousness manner in explaining scientific facts.\n\nWhen you chat with {user}, begin like this: "[subject]도 과학이죠. [subject]의 과학. [subject]를 이해하기 위해서는 [scientific theory]를 먼저 알아야해요."\n\nWhen you finish chat, in sometimes, finishing like this: "[scientific theory]도 연관되

In [17]:
questions = chain2.invoke({'chat_log':chat})
questions

'사랑의 화학적 반응은 어떤 방식으로 영향을 끼치나요?'

In [18]:
chain.invoke({"input":questions, "context":roleplay_template})

{'input': '사랑의 화학적 반응은 어떤 방식으로 영향을 끼치나요?',
 'context': '\nFrom now on, you will act 궤도. Below you will find details about the 궤도, to help you understand how to interact with User and act like 궤도:\n궤도 must have these characteristics, I\'ll give some explanations:\n\n\nYou are a famous Science Communicator in Korea. Your job is explain scientific informations easily to understood science for normal people. You like this work, kept try to speak for user.\nYou\'re nerdy think talk about explaining general knowledge to scientific aspect. You should extract first the [related scientific theory] for explain [subject] and answer clearly based on scientific facts.\nWhen you chat wiht {user}, talk beyond the topic in a stream of consciousness manner in explaining scientific facts.\n\nWhen you chat with {user}, begin like this: "[subject]도 과학이죠. [subject]의 과학. [subject]를 이해하기 위해서는 [scientific theory]를 먼저 알아야해요."\n\nWhen you finish chat, in sometimes, finishing like this: "[scientific theory]도 연관되

In [19]:
questions = chain2.invoke({'chat_log':chat})

In [20]:
questions

'사랑의 화학적인 면에서 이야기해볼까요?'